In [1]:
from stable_baselines3 import A2C
from stable_baselines3.common.policies import MultiInputActorCriticPolicy
import cv2
from gymnasium.wrappers import RecordEpisodeStatistics
from deepface import DeepFace
import gymnasium as gym
from gymnasium import spaces
import numpy as np
import librosa
from pydub import AudioSegment
import torch
import torch.nn as nn
from stable_baselines3.common.torch_layers import BaseFeaturesExtractor
from sklearn.preprocessing import MinMaxScaler

from transformers import pipeline
import whisper

sentiment_model = pipeline("sentiment-analysis", model="nlptown/bert-base-multilingual-uncased-sentiment")
emotion_model = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")
topic_model = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

Device set to use cuda:0
Device set to use cuda:0
Device set to use cuda:0


In [2]:


def extract_frames_gray_scale(video_path):
    cap = cv2.VideoCapture(video_path)
    print("actor cap",cap)
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    print("actor fps: ",fps)
    
    # Pobranie wymiarów pierwszej klatki
    ret, first_frame = cap.read()
    if not ret:
        cap.release()
        raise ValueError("Nie można odczytać pierwszej klatki wideo!")

    # height, width, channels = first_frame.shape  # Pobranie wymiarów obrazu
    print("actor 1st frame",first_frame.shape)
    # Konwersja pierwszej klatki na grayscale i dodanie trzeciego wymiaru
    first_frame_gray = cv2.cvtColor(first_frame, cv2.COLOR_BGR2GRAY)
    frames = [np.expand_dims(first_frame_gray, axis=-1)]  # dodajemy kanał, by mieć (wysokość, szerokość, 1)
    frame_idx = fps  # Pierwsza klatka już dodana
    while cap.isOpened():
        cap.set(cv2.CAP_PROP_POS_FRAMES, frame_idx)
        ret, frame = cap.read()
        if not ret:
            break
        # Konwersja do grayscale i dodanie kanału
        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        frames.append(np.expand_dims(gray_frame, axis=-1))  # dodajemy kanał, by mieć (wysokość, szerokość, 1)
        frame_idx += fps
    
    cap.release()
    return frames


def extract_frames_rgb(video_path):
    cap = cv2.VideoCapture(video_path)
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    print("critic fps: ",fps)
    
    # Pobranie pierwszej klatki wideo
    ret, first_frame = cap.read()
    print("critic 1st frame",first_frame.shape)
    if not ret:
        cap.release()
        raise ValueError("Nie można odczytać pierwszej klatki wideo!")
    
    frames_rgb = [cv2.cvtColor(first_frame, cv2.COLOR_BGR2RGB)]
    

    frame_idx = fps  # Pomijamy pierwszą klatkę, bo już ją dodaliśmy
    while cap.isOpened():
        cap.set(cv2.CAP_PROP_POS_FRAMES, frame_idx)
        ret, frame = cap.read()
        if not ret:
            break
        frames_rgb.append(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))  # Konwersja BGR → RGB
        frame_idx += fps
    
    cap.release()
    return frames_rgb


#nagroda, gotowe clpy

def calculate_video_score(frames, clip_length, skip):
    emotion_scores = []
    

    
    emotion_intensity = {
                'happy': 0.7,
                'surprise': 0.95,
                'angry': 0.9,
                'sad': 0.4,
                'neutral': 0.1,
                'disgust': 0.9,
                'face_confidence': 0.1,
                'fear': 0.95
            }
    
    
    # emotion_intensity = {
    #             'happy': 0.7,
    #             'surprise': -0.95,
    #             'angry': -0.9,
    #             'sad': -0.4,
    #             'neutral': -0.1,
    #             'disgust': -0.9,
    #             'face_confidence': -0.1,
    #             'fear': -0.95
    #         }
    
    for frame in frames:
            try:
                analysis = DeepFace.analyze(frame, actions=['emotion'], enforce_detection=False, detector_backend='opencv')
        
                emotion_values = analysis[0]['emotion']
                
                normalized_emotions = {emotion: value / 100 for emotion, value in emotion_values.items()}
                
                sorted_emotions = sorted(normalized_emotions.items(), key=lambda x: x[1], reverse=True)
                
        
                threshold = 0.5
        
                top_3_emotions = [(emotion, value) for emotion, value in sorted_emotions[:3] if value > threshold]
        
                # Jeśli mamy mniej niż 3 emocje powyżej progu, wstawiamy 0 dla brakujących
                while len(top_3_emotions) < 3:
                    top_3_emotions.append(('none', 0))
        
                # Obliczanie średniej wartości emocji z wagami
                emotion_weighted_average = 0.0
                for i, (emotion, value) in enumerate(top_3_emotions):
                    weight = emotion_intensity.get(emotion, 0)
                    if i == 0:
                        emotion_weighted_average += (value * weight * 0.60)
                    elif i == 1:
                        emotion_weighted_average += (value * weight * 0.25)
                    elif i == 2:
                        emotion_weighted_average += (value * weight * 0.15)
                
                emotion_scores.append(emotion_weighted_average)
    
            except Exception as e:
                emotion_scores.append(0)
    clip_emotion_score = create_clips_from_frames(emotion_scores, clip_length, skip)        
    return clip_emotion_score


def create_actor_video_clips(video_path, clip_length, skip):
    frames = extract_frames_rgb(video_path)
    return create_clips_from_frames(frames, clip_length, skip)
    
   

#pomocnicze    
def generate_final_video(selected_clips ,video_path ,output_folder="C:\\Users\\pwdlp\\OneDrive\\Pulpit\\Clips"):
        """Tworzy osobne filmy dla każdego wybranego zakresu czasowego."""
    
        if not selected_clips:
            print("⚠️ Brak wybranych fragmentów! Nie można wygenerować filmów.")
            return
    
        # Otwieramy oryginalne wideo
        cap = cv2.VideoCapture(video_path)
        fps = int(cap.get(cv2.CAP_PROP_FPS))
    
        if not cap.isOpened():
            print("❌ Nie udało się otworzyć pliku wideo!")
            return
    
        height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    
        # Używamy kodeku 'mp4v' dla plików .mp4
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    
        # Przetwarzanie każdej krotki w selected_clips

        for clip_index, (start_sec, end_sec) in enumerate(selected_clips):
            # Ścieżka do zapisu pliku wyjściowego
            output_path = f"{output_folder}\\clip_{clip_index}.mp4"
    
            # Inicjalizacja VideoWriter dla nowego pliku
            out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))
    
            if not out.isOpened():
                print(f"❌ Nie udało się utworzyć pliku wyjściowego: {output_path}")
                continue
    
            # Przeliczenie sekund na klatki
            start_frame = int(start_sec * fps)
            end_frame = int(end_sec * fps)
    
            # Ustawienie pozycji na początkową klatkę
            cap.set(cv2.CAP_PROP_POS_FRAMES, start_frame)
    
            # Zapis klatek w zakresie
            while cap.get(cv2.CAP_PROP_POS_FRAMES) < end_frame:
                ret, frame = cap.read()
                if not ret:
                    print(f"❌ Błąd: Nie udało się odczytać klatki w klipie {clip_index}!")
                    break
                out.write(frame)
    
            # Zamknięcie pliku wyjściowego
            out.release()
            print(f"✅ Klip {clip_index} zapisano jako: {output_path}")
    
        # Zwolnienie zasobów
        cap.release() 
     
#        
        
def extract_audio(video_path, sample_rate=16000):

    # 1. Wczytaj plik wideo za pomocą PyDub
    audio = AudioSegment.from_file(video_path)

    # 2. Jeśli częstotliwość próbkowania nie jest zgodna z docelową, wykonaj resampling
    if audio.frame_rate != sample_rate:
        audio = audio.set_frame_rate(sample_rate)

    # 3. Przekonwertuj dźwięk do numpy array
    audio_array = np.array(audio.get_array_of_samples(), dtype=np.float32)

    # 4. Jeśli dźwięk jest stereo, przekształć go na mono
    if audio.channels > 1:
        audio_array = audio_array.reshape(-1, audio.channels).mean(axis=1)

    # 5. Normalizuj dźwięk do zakresu [-1, 1]
    audio_array = audio_array / (2 ** (audio.sample_width * 8 - 1))


    return audio_array   

def create_clips_from_frames(frames, clip_length, skip=1):
    return [frames[start:start + clip_length] for start in range(0, len(frames) - clip_length + 1, skip)]

    

funkcja do pobierania i oceny właściwosci dźwięku


In [3]:



def extract_audio_metrics(audio_array, sr):
    """
    Wyciąga i normalizuje metryki audio, zwracając słownik.
    """

    T = len(audio_array) / sr

    # Minimalny czas trwania onsetu (w sekundach)
    min_onset_duration = 0.1  # 100 ms

    # Oblicz maksymalną liczbę onsetów
    max_onsets = T / min_onset_duration

    # Wyciąganie metryk audio
    rms = librosa.feature.rms(y=audio_array)[0]
    spectral_centroid = librosa.feature.spectral_centroid(y=audio_array, sr=sr)[0]
    energy = np.sum(audio_array**2) / len(audio_array)
    onset_frames = librosa.onset.onset_detect(y=audio_array, sr=sr, units='frames')
    zcr = librosa.feature.zero_crossing_rate(y=audio_array)[0]
    silence_ratio = np.mean(np.abs(audio_array) < 0.01)

    # Normalizacja metryk do zakresu [0, 1]
    rms_normalized = np.mean(rms)  # Już w zakresie [0, 1]
    spectral_centroid_normalized = np.mean(spectral_centroid) / (sr / 2)  # Znormalizowane
    energy_normalized = energy  # Już w zakresie [0, 1]
    onset_count_normalized = len(onset_frames) / max_onsets  # Znormalizowane
    zcr_normalized = np.mean(zcr) / 0.5  # Znormalizowane
    silence_score = 1 - silence_ratio  # Już w zakresie [0, 1]

    # Wyciąganie metryk i tworzenie słownika
    metrics = {
        "rms_mean": rms_normalized ,
        "spectral_std": spectral_centroid_normalized,
        "energy_log": energy_normalized,
        "onset_count": onset_count_normalized,
        "zcr_mean": zcr_normalized,
        "silence_score": silence_score
    }

    return metrics


def calculate_clip_audio_score(metrics):
    weights = {
        "rms_mean": 0.3,
        "spectral_std": 0.05,
        "energy_log": 0.2,
        "onset_count": 0.4,
        "zcr_mean": 0.03,
        "silence_score": 0.02
    }
    score = sum(metrics[key] * weight for key, weight in weights.items())
    
    return score

def split_audio_into_clips(audio_array, sample_rate, clip_duration, total_duration, skip):
    clips = [
        (
            audio_array[int(start * sample_rate):int((start + clip_duration) * sample_rate)],
            sample_rate
        )
        for start in np.arange(0, total_duration - clip_duration + skip, skip)
        if (start + clip_duration) <= total_duration
    ]
    return clips

def calculate_audio_score(video_path, clip_duration, skip, total_duration, sample_rate=22050):
    # 1. Ekstrahuj cały dźwięk
    audio_array = extract_audio(video_path, sample_rate)

    # 2. Przygotuj listę przedziałów czasowych do analizy
    clip_times = split_audio_into_clips(audio_array, sample_rate, clip_duration, total_duration, skip)

    # 3. Oblicz score dla każdego fragmentu
    metrics = [extract_audio_metrics(*clip) for clip in clip_times]
    scores = [calculate_clip_audio_score(metrics) for metrics in metrics]

    return np.array(scores)


def create_actor_audio_clips(video_path, clip_duration, skip, total_duration, sample_rate=22050):
    audio_array = extract_audio(video_path, sample_rate)
    clip_times = split_audio_into_clips(audio_array, sample_rate, clip_duration, total_duration, skip)
    metrics = [list(extract_audio_metrics(*clip).values()) for clip in clip_times]
    return np.array(metrics)
    

    
    

funkcje do przetwarzania dźwięku na tekst i jego oceny

In [4]:
def audio_to_text(audio_array, sample_rate=16000, device="cuda"):
    if not isinstance(audio_array, np.ndarray):
        raise ValueError("audio_array should be a numpy array")

    # Upewnij się, że częstotliwość próbkowania jest zgodna z wymaganiami Whispera
    if sample_rate != 16000:
        raise ValueError("Whisper expects audio with a sample rate of 16000 Hz")
    
    

    # Załaduj model Whisper i przenieś go na GPU
    model = whisper.load_model("base").to(device)

    result = model.transcribe(audio_array, fp16=False)

    return result["text"]


#do przekminienia (najlepiej opierdolic to tą drugą funnkcją do podziału zeby była jedna)
def split_text_by_time(text, segment_duration=20):
    """
    Funkcja dzieli tekst na fragmenty o zadanej długości w sekundach
    (segment_duration to długość w sekundach).
    """
    # W tym przykładzie zakładajmy, że mamy podzielony tekst na linie
    # Możesz podzielić w zależności od swojej struktury
    lines = text.split("\n")
    segments = []
    
    current_segment = []
    current_duration = 0

    # Załóżmy, że każda linia trwa około 2 sekund (możesz dostosować)
    line_duration = 2 

    for line in lines:
        current_segment.append(line)
        current_duration += line_duration
        if current_duration >= segment_duration:
            segments.append(" ".join(current_segment))
            current_segment = []
            current_duration = 0

    # Dodajemy ostatni fragment, jeśli pozostały linie
    if current_segment:
        segments.append(" ".join(current_segment))

    return segments


def analyze_text_with_roberta(text):
    # Analizowanie sentymentu
    sentiment = sentiment_model(text)
    
    # Analiza emocji
    emotion_result = emotion_model(text, candidate_labels=["joy", "sadness", "anger", "fear", "surprise", "disgust"])
    
    # Analiza tematu
    topics = ["politics", "sports", "entertainment", "technology", "science", "health"]
    topic_result = topic_model(text, candidate_labels=topics)
    
    # Możesz dodać inne analizy, np. złożoność tekstu itp.

    # Wagi dla wyników analizy
    weights = {
        "sentiment": 0.2,
        "emotion": 0.3,
        "topic": 0.2
        # Dodaj wagi dla innych analiz
    }

    # Przypisujemy wagi do wyników i obliczamy końcowy score
    score = (
        weights["sentiment"] * sentiment[0]['score'] +
        weights["emotion"] * emotion_result['scores'][0] +
        weights["topic"] * topic_result['scores'][0]
    )

    return score, [sentiment, emotion_result, topic_result]


def calculate_scores_for_video(video_path, segment_duration=20):
    audio, duration = extract_audio(video_path)
    text = audio_to_text(audio)
    segments = split_text_by_time(text, segment_duration)
    result_metrics = [analyze_text_with_roberta(segment) for segment in segments]
    scores, metrics = zip(*result_metrics)
    return scores, metrics

In [5]:
def skewed_gaussian_weights(length):
    """
    Generuje wagi, które:
    - Od 0 do 0.6 rosną szybko, nieliniowo (kwadratowo), z zaokrągleniem w okolicach środka.
    - Od 0.6 do 0.8 bardzo powoli spadają (kwadratowo).
    - Od 0.8 do 1 gwałtownie spadają liniowo do 0.
    """
    # Tworzymy zakres x od 0 do 1
    x = np.linspace(0, 1, length)
    
    # Inicjalizacja wag
    weights = np.zeros(length)
    
    # 1. Od 0 do 0.6: szybki wzrost nieliniowy (kwadratowy) z zaokrągleniem
    mask_rise = x <= 0.6
    x_rise = x[mask_rise] / 0.6  # Normalizacja do zakresu [0, 1]
    weights[mask_rise] = 0.3 + 0.7 * (x_rise ** 2)  # Kwadratowy wzrost
    
    # 2. Od 0.6 do 0.8: bardzo powolny spadek nieliniowy (kwadratowy)
    mask_gentle_fall = (x > 0.6) & (x <= 0.8)
    x_fall = (x[mask_gentle_fall] - 0.6) / 0.2  # Normalizacja do zakresu [0, 1]
    weights[mask_gentle_fall] = 1.0 - 0.5 * (x_fall ** 2)  # Kwadratowy spadek
    
    # 3. Od 0.8 do 1: gwałtowny spadek liniowy do 0
    mask_sharp_fall = x > 0.8
    x_sharp_fall = (x[mask_sharp_fall] - 0.8) / 0.2  # Normalizacja do zakresu [0, 1]
    weights[mask_sharp_fall] = 0.5 - 0.5 * x_sharp_fall  # Liniowy spadek
    
    # Normalizacja wag
    weights = weights / weights.sum()
    
    return weights

class VideoClipEnv(gym.Env):
    def __init__(self, clips, audio_metrics, clip_length, video_scores, audio_scores):
        super(VideoClipEnv, self).__init__()
        
        #operujemy na liczbie klatek ponieważ jedna klatka tożsama jest z jedną sekundą w tym podejściu
        self.clips = clips  # Lista klipów (każdy klip to tablica klatek)
        self.audio_metrics = audio_metrics  # Lista metryk dźwięku (dla każdego klipu)
        # self.text_metrics = text_metrics  # Lista metryk tekstu (dla każdego klipu
        self.total_number_of_clips = len(self.clips)
        
        self.video_emotion_scores = video_scores
        self.audio_emotion_scores = audio_scores
        # self.text_emotion_scores = text_scores

        

        # Przestrzeń akcji: 0 (pomiń), 1 (zachowaj)
        self.action_space = spaces.Discrete(2)

        self.clip_length = clip_length

        self.observation_space = spaces.Dict({
            "video": spaces.Box(low=0, high=255, shape=(self.clip_length, 1080, 1920, 3), dtype=np.uint8),  # Klipy
            "audio_metrics": spaces.Box(low=0, high=1, shape=(self.clip_length, 6), dtype=np.float32),  # Metryki dźwięku
            # "text_metrics": spaces.Box(low=0, high=1, shape=(self.clip_length, 3), dtype=np.float32),  # Metryki tekstu
        })
        
        self.step_length = 1

        self.current_step = 0
        
    def reset(self, seed=None, options=None):
        super().reset(seed=seed)
        self.current_step = 0

        observation = self._get_observation()
        info = {}  # Dodaj pusty słownik info
        return observation, info

    def step(self, action):
        
        observation = self._get_observation()
        
        reward = self._calculate_reward() if action == 1 else 0

        done = self.current_step > self.total_number_of_clips
        
        self.current_step += self.step_length 

    
        return observation, reward, done, False, {}


    def _get_observation(self):
        
        clip = self.clips[self.current_step]
        audio = self.audio_metrics[self.current_step]
        # text = self.text_metrics[self.current_step]
    
        return {
            "video": np.array(clip, dtype=np.uint8),
            "audio_metrics": np.array(audio, dtype=np.float32),
            # "text_metrics": np.array(text, dtype=np.float32),
        }

    def _calculate_reward(self):
        emotion_score = self._detect_emotion()
        audio_score = self._calculate_audio_score()

        reward = emotion_score * 0.3 + audio_score * 0.7
    
        return reward
    

    def render(self, mode='human'):
        # print(f"Step Render: {self.current_step}, Selected Clips: {self.selected_clips}")
        # print(f"Emotions: {self._detect_emotion([self.frames[self.current_step]])}")
        pass

    def close(self):
        torch.cuda.empty_cache()
    
    #funkcje dodatkowe
    def _detect_emotion(self):
        scores = np.array(self.video_emotion_scores[self.current_step])
    
        weights = skewed_gaussian_weights(len(scores))

        return np.sum(scores * weights)
    def _calculate_audio_score(self):
        score = np.array(self.audio_emotion_scores[self.current_step])

        
        return np.sum(score)



In [6]:


class CustomEfficientNetCNN(BaseFeaturesExtractor):
    def __init__(self, observation_space, features_dim=1024):
        super(CustomEfficientNetCNN, self).__init__(observation_space, features_dim)
        
        # Warstwa wejściowa
        self.initial_conv = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, stride=2, padding=1, bias=False),  # 1920x1080 -> 960x540
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
        )
        
        # Bloki MBConv (inspirowane EfficientNet)
        self.conv_blocks = nn.Sequential(
            self._make_mbconv_block(32, 16, stride=1),  # 960x540 -> 960x540
            self._make_mbconv_block(16, 24, stride=2),  # 960x540 -> 480x270
            self._make_mbconv_block(24, 40, stride=2),  # 480x270 -> 240x135
            self._make_mbconv_block(40, 80, stride=2),  # 240x135 -> 120x68
            self._make_mbconv_block(80, 112, stride=2),  # 120x68 -> 60x34
            self._make_mbconv_block(112, 192, stride=2),  # 60x34 -> 30x17
        )
        
        # Warstwa spłaszczająca
        self.flatten = nn.Flatten()
        
        # Warstwa liniowa do dopasowania do wymaganego wymiaru cech (1024)
        self.fc = nn.Linear(192 * 30 * 17, features_dim)
    
    def _make_mbconv_block(self, in_channels, out_channels, stride):
        # Blok MBConv z rozszerzeniem i redukcją kanałów
        return nn.Sequential(
            nn.Conv2d(in_channels, in_channels * 6, kernel_size=1, stride=1, padding=0, bias=False),  # Rozszerzenie
            nn.BatchNorm2d(in_channels * 6),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels * 6, in_channels * 6, kernel_size=3, stride=stride, padding=1, groups=in_channels * 6, bias=False),  # Depthwise
            nn.BatchNorm2d(in_channels * 6),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels * 6, out_channels, kernel_size=1, stride=1, padding=0, bias=False),  # Redukcja
            nn.BatchNorm2d(out_channels),
        )
    
    def forward(self, observations):
        # Pobierz dane wideo
        video = observations["video"]  # Kształt: (batch_size, liczba_klatek, wysokość, szerokość, kanały)
        
        # Przekształć kształt na (batch_size * liczba_klatek, kanały, wysokość, szerokość)
        batch_size, num_frames, height, width, channels = video.shape
        video = video.view(batch_size * num_frames, height, width, channels)  # (batch_size * num_frames, height, width, channels)
        video = video.permute(0, 3, 1, 2)  # (batch_size * num_frames, channels, height, width)
        
        # Przetwórz klatki wideo przez warstwę wejściową
        x = self.initial_conv(video)
        
        # Przetwórz przez bloki MBConv
        x = self.conv_blocks(x)
        
        # Spłaszcz i przetwórz przez warstwę liniową
        x = self.flatten(x)
        
        # Przywróć oryginalny kształt batcha
        x = x.view(batch_size, num_frames, -1)  # (batch_size, num_frames, features_dim)
        
        # Uśrednij cechy po klatkach
        x = x.mean(dim=1)  # (batch_size, features_dim)
        
        return self.fc(x)

In [7]:
test_path = "C:\\Users\\pwdlp\\OneDrive\\Pulpit\\Vods\\sperma.mp4"
clip_duration = 10
skip = 1
sample_rate = 22050

frames  = extract_frames_rgb(test_path)


#video clips and scores
clips_for_actor = create_clips_from_frames(frames, clip_duration, skip)
total_duration = len(clips_for_actor)

emotion_score = calculate_video_score(frames, clip_duration, skip)

#tekst

# text_score, actor_text =  calculate_scores_for_video(test_path, clip_duration)

#audio clips and scores
audio_score = calculate_audio_score(test_path, clip_duration, skip, total_duration, sample_rate=sample_rate)
audio_for_actor = create_actor_audio_clips(test_path, clip_duration, skip, total_duration, sample_rate=sample_rate)

# print(audio_for_actor[0])



critic fps:  25
critic 1st frame (1080, 1920, 3)


In [65]:


torch.cuda.empty_cache()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


policy_kwargs = dict(
    features_extractor_class=CustomEfficientNetCNN,
    features_extractor_kwargs=dict(features_dim=1024),
)


env = VideoClipEnv(clips_for_actor, audio_for_actor, clip_duration, emotion_score, audio_score)
env = RecordEpisodeStatistics(env)

model = A2C(
    MultiInputActorCriticPolicy,
    env=env,
    policy_kwargs=policy_kwargs,
    verbose=1,
    n_steps=30,
    ent_coef=0.01,
    learning_rate=0.001,
    gamma=0.99,
    device=device,
)
 
model.learn(total_timesteps=100)

Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


KeyboardInterrupt: 

In [ ]:

obs, _ = env.reset()
done = False
selected_clips = []  # Lista na wybrane klipy

env_original = env.env

while not done:
    action, _states = model.predict(obs, deterministic=True)  # Wybieramy akcję
    next_obs, reward, done, truncated, info = env.step(action)  # Wykonaj krok w środowisku
    if action == 1:
        selected_clips.append((env_original.current_step, env_original.current_step + env_original.clip_length))

    obs = next_obs

env.close()



testowanie